# Analysis notebook for the quals report

The main point of this document is to put together all the pretty graphs and tables that prove our hypotheses and to document how we got them.

## Obtaining main timing results and statistics
The data we're working was generated from swizzleflow commit `49c8255170`.

We have the following files

1. `results/2020-04-26-timings-for-initial-quals-eval`
   - `env RUSTFLAGS="-g" cargo build --features stats --release `
   - ```bash
for i in specs/swinv_like/*/*.json                              (master*)
do
./target/release/swizzleflow -a $i | tee -a results/2020-04-26-timings-for-inital-quals-eval
done
```

2. `results/2020-04-27-2d-stencil-5-timings`
   - ```bash
for i in specs/swinv_like_big/*/*.json                           (master)
do                       
./target/release/swizzleflow -a $i | tee results/2020-04-27-2d-stencil-5-timings
done
```
   - Get yourself some coffee while that's going

3. `results/2020-04-27-stats-for-swinv-specs-for-quals`
   - Recompile with `env RUSTFLAGS="-g" cargo build --features stats --release`
   - Run the same `for` loop in point 1, but outputing to a different file. These experiments will be slower due to statistic collection, but will give more info on what's going on

4. For `results/2020-04-27-swizzleflow-comparisons.txt`, see that file for vague mumblings about the changes we had to make to their code to get all the data we caned about, or, like, call me if I haven't documented it better by then

## Obtaining results for the one point experiments
1. `git checkout one_point_test`, which has code modified to track only one point
2. `env RUSTFLAGS="-g" cargo build --features stats --release` to comple
3. We used small specs to make sure we wouldn't have issues with search performance.
4. And so run `./target/release/swizzleflow -m mats_one -a ./specs/1d-conv-16x3.json ./specs/trove-16x3.json | tee results/2020-04-28-small-specs-one-point-tracked-stats ` on the branch
5. `git checkout master`, recompile
6. `./target/release/swizzleflow -a ./specs/1d-conv-16x3.json ./specs/trove-16x3.json | tee results/2020-04-28-small-specs-two-points-tracked-stats` to get the results

In [ ]:
# initial setup
import sys
sys.path.append("../analysis")

In [2]:
import parsing
import extraction

from parsing import parse_file
from extraction import humanize_names, expand_target_checks, pull_spec_in

In [3]:
%matplotlib widget

In [4]:
import pandas as pd
import numpy as np

import itertools

import matplotlib.pyplot as plt

In [5]:
def fetch(dataset):
    return humanize_names(parse_file(f"../results/{dataset}"))

In [6]:
## Pruning utility experiments
one_point_raw = fetch("2020-04-28-small-specs-one-point-tracked-stats")
two_points_raw = fetch("2020-04-28-small-specs-two-points-tracked-stats")

In [7]:
def map_frames(frames, f):
    return {k: f(v) for k, v in frames.items()}


In [8]:
cols_to_keep = ['name', 'tested', 'pruned', 'failed', 'found', 'continued']
one_point = extraction.search_stats(one_point_raw)
two_points = extraction.search_stats(two_points_raw)
one_point = map_frames(one_point,
                       lambda v: v[cols_to_keep])
two_points = map_frames(two_points,
                        lambda v: v[cols_to_keep])

In [9]:
n_solutions = map_frames(two_points, lambda v: v['found'].sum())

In [10]:
def points_transform(points, spec, frame):
    mapping = {'1d-conv-16x3': 'Convolution', 'trove-16x3': 'Trove'}
    df = frame[1:-1].copy()
    df['Spec'] = mapping[spec]
    df['Locations'] = points
    df['n_solutions'] = n_solutions[spec]
    df.loc[df.index[-1], 'pruned'] = frame['failed'].iloc[-1]
    df.loc[df.index[-1], 'continued'] = n_solutions[spec]
    return df
frames = [points_transform('One', k, v) for k, v in one_point.items()]
frames.extend([points_transform('Two', k, v) for k, v in two_points.items()])
all_data = pd.concat(frames)
all_data

,name,tested,pruned,failed,found,continued,Spec,Locations,n_solutions
1,reg_select,38,31,0,0,7,Convolution,One,2
2,col_xforms,2268,0,0,0,2268,Convolution,One,2
3,col_rots,61236,61234,0,0,2,Convolution,One,2
4,cond_keep,76,18,56,0,2,Convolution,One,2
1,row_xforms,24,0,0,0,24,Trove,One,2
2,row_rots,72,0,0,0,72,Trove,One,2
3,col_xforms,23328,0,0,0,23328,Trove,One,2
4,col_rots,629856,629854,0,0,2,Trove,One,2
5,row_xforms,48,0,0,0,48,Trove,One,2
6,row_rots,144,142,0,0,2,Trove,One,2


In [11]:
all_data.index = all_data.index.map(lambda n: f"s{n+2}")

In [12]:
# Ripped frov https://github.com/pandas-dev/pandas/issues/23955
def multiindex_pivot(df, index = None, columns = None, values = None):
    # https://github.com/pandas-dev/pandas/issues/23955
    output_df = df.copy(deep = True)
    if index is None:
        names = list(output_df.index.names)
        output_df = output_df.reset_index()
    else:
        names = index
    output_df = output_df.assign(tuples_index = [tuple(i) for i in output_df[names].values])
    if isinstance(columns, list):
        output_df = output_df.assign(tuples_columns = [tuple(i) for i in output_df[columns].values])  # hashable
        output_df = output_df.pivot(index = 'tuples_index', columns = 'tuples_columns', values = values) 
        output_df.columns = pd.MultiIndex.from_tuples(output_df.columns, names = columns)  # reduced
    else:
        output_df = output_df.pivot(index = 'tuples_index', columns = columns, values = values)    
    output_df.index = pd.MultiIndex.from_tuples(output_df.index, names = names)
    return output_df

In [13]:
plot_df = all_data[['Locations', 'Spec', 'n_solutions', 'continued']].copy(deep=True)
plot_df.index.name = "Statement"
plot_df.set_index('Spec', append=True, inplace=True)
plot_df = plot_df.reorder_levels([1, 0])
plot_df['in_solution'] = plot_df['n_solutions'] / plot_df['continued']
plot_df.drop(['n_solutions', 'continued'], inplace=True, axis=1)
pivoted = multiindex_pivot(plot_df, columns='Locations', values='in_solution')
fig, axs = plt.subplots(1, 2)
fig.suptitle("Usefulness of subtree exploration with N elements")
for ax, groups in zip(axs, pivoted.reset_index(level='Spec').groupby('Spec')):
    name, data = groups
    data.plot(kind='line', style='o-', ax=ax)
    ax.set_title(name)
axs[0].set_ylabel("% of states in solution")
None

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig.savefig('/tmp/percentage-usefulness.pdf')

In [15]:
table_df = all_data[['Locations', 'Spec', 'tested', 'continued']].copy(deep=True)
table_df.index.name = "Statement"
table_df.set_index('Spec', append=True, inplace=True)
table_df = table_df.reorder_levels([1, 0])
table_df.rename(columns={'tested': 'Tested', 'continued': 'Unpruned'}, inplace=True)
table_pivoted = multiindex_pivot(table_df, columns='Locations',
                                 values=['Tested', 'Unpruned'])
table_pivoted = table_pivoted.reorder_levels([1, 0], axis=1)
table_pivoted

Locations                 One    Two      One      Two
                       Tested Tested Unpruned Unpruned
Spec        Statement                                 
Convolution s3             38     38        7        2
            s4           2268    648     2268        2
            s5          61236     54        2        2
            s6             76     76        2        2
Trove       s3             24     24       24        2
            s4             72      6       72        2
            s5          23328    648    23328        2
            s6         629856     54        2        2
            s7             48     48       48        2
            s8            144      6        2        2

In [16]:
c = table_pivoted.columns
table = table_pivoted[[c[0], c[2], c[1], c[3]]]
table

Locations                 One             Two         
                       Tested Unpruned Tested Unpruned
Spec        Statement                                 
Convolution s3             38        7     38        2
            s4           2268     2268    648        2
            s5          61236        2     54        2
            s6             76        2     76        2
Trove       s3             24       24     24        2
            s4             72       72      6        2
            s5          23328    23328    648        2
            s6         629856        2     54        2
            s7             48       48     48        2
            s8            144        2      6        2

In [17]:
print(table.loc['Trove'].to_latex())

\begin{tabular}{lrrrr}
\toprule
Locations & \multicolumn{2}{l}{One} & \multicolumn{2}{l}{Two} \\
{} &  Tested & Unpruned & Tested & Unpruned \\
Statement &         &          &        &          \\
\midrule
s3        &      24 &       24 &     24 &        2 \\
s4        &      72 &       72 &      6 &        2 \\
s5        &   23328 &    23328 &    648 &        2 \\
s6        &  629856 &        2 &     54 &        2 \\
s7        &      48 &       48 &     48 &        2 \\
s8        &     144 &        2 &      6 &        2 \\
\bottomrule
\end{tabular}



In [18]:
print(table.to_latex())

\begin{tabular}{llrrrr}
\toprule
      & Locations & \multicolumn{2}{l}{One} & \multicolumn{2}{l}{Two} \\
      & {} &  Tested & Unpruned & Tested & Unpruned \\
Spec & Statement &         &          &        &          \\
\midrule
Convolution & s3 &      38 &        7 &     38 &        2 \\
      & s4 &    2268 &     2268 &    648 &        2 \\
      & s5 &   61236 &        2 &     54 &        2 \\
      & s6 &      76 &        2 &     76 &        2 \\
Trove & s3 &      24 &       24 &     24 &        2 \\
      & s4 &      72 &       72 &      6 &        2 \\
      & s5 &   23328 &    23328 &    648 &        2 \\
      & s6 &  629856 &        2 &     54 &        2 \\
      & s7 &      48 &       48 &     48 &        2 \\
      & s8 &     144 &        2 &      6 &        2 \\
\bottomrule
\end{tabular}



## Trove scaling with problem size

In [19]:
raw_data = fetch("2020-04-26-timings-for-initial-quals-eval")

In [20]:
import timing

In [21]:
swflow_times_raw = timing.process(raw_data)

In [22]:
swflow_times_raw.head()

,matrix time,search time,total
spec,,,
l1/1d-conv,2.573750,0.007135,2.580885
l1/1d-stencil,0.009763,0.003000,0.012764
l1/2d-stencil-3,9.311759,0.002135,9.313894
l1/mult-32-with-4,0.002930,0.002111,0.005041
l1/trove-crc-1,0.011006,0.000161,0.011167


In [23]:
swflow_times = swflow_times_raw.reset_index()[['spec', 'total']].copy()
swflow_times.head()

,spec,total
0,l1/1d-conv,2.580885
1,l1/1d-stencil,0.012764
2,l1/2d-stencil-3,9.313894
3,l1/mult-32-with-4,0.005041
4,l1/trove-crc-1,0.011167


In [24]:
swflow_times['Level'] = swflow_times['spec'].map(lambda s: int(s[1]))

In [25]:
swflow_times['Problem'] = swflow_times['spec'].map(lambda s: s[3:])

In [26]:
trove_times = swflow_times[(swflow_times['Level'] == 3)\
            & (swflow_times['Problem'].map(lambda s: s.startswith("trove-crc-")))]\
            .copy()

In [27]:
trove_times['Terms'] = trove_times['Problem']\
                        .map(lambda s: int(s[-1]) * 32)

In [28]:
trove_times = trove_times.drop(columns=['spec', 'Level', 'Problem'])\
                        .rename(columns={'total': 'Time'})
trove_times = trove_times.set_index('Terms')

In [29]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
trove_times.plot(kind='line', style='o-', ax=ax,
                 legend=False,
                 xlim=[30,170],
                 title='Synthesis scalability with problem size')
ax.set_ylabel('CRC Trove synthesis time (s)')
fig.savefig('/tmp/trove-times.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
search_stats = pull_spec_in(extraction.search_stats(raw_data))
search_stats = search_stats[search_stats['spec']
                            .map(lambda s: s.endswith("trove-crc-3"))].copy()
search_stats = search_stats.reset_index().drop(columns='index')

In [31]:
extraction.compute_basis_size(search_stats)

In [32]:
search_stats[['name', 'spec', 'basis_size']]

,name,spec,basis_size
0,load_rep,l1/trove-crc-3,1.0
1,row_xforms_no_group,l1/trove-crc-3,30.0
2,row_rots_no_group,l1/trove-crc-3,3.0
3,col_xforms_no_group,l1/trove-crc-3,992.0
4,col_rots_no_group,l1/trove-crc-3,32.0
5,row_xforms_no_group,l1/trove-crc-3,30.0
6,row_rots_no_group,l1/trove-crc-3,3.0
7,(last),l1/trove-crc-3,1.0
8,load_rep,l3/trove-crc-3,inf
9,row_xforms,l3/trove-crc-3,30.0


In [33]:
# Per amove
xforms_1 = 992
rots_1 = 32
xforms_2 = 1347
rots_2 = 58
(xforms_1 * rots_1, xforms_2 * rots_2)

(31744, 78126)

## Comparing to Swizzle Inventor

In [34]:
swflow_times['spec']

0            l1/1d-conv
1         l1/1d-stencil
2       l1/2d-stencil-3
3     l1/mult-32-with-4
4        l1/trove-crc-1
5        l1/trove-crc-2
6        l1/trove-crc-3
7        l1/trove-crc-4
8        l1/trove-crc-5
9     l1/trove-cr_sum-1
10    l1/trove-cr_sum-2
11    l1/trove-cr_sum-3
12    l1/trove-cr_sum-4
13    l1/trove-cr_sum-5
14    l1/trove-cr_sum-7
15       l1/trove-rcr-1
16       l1/trove-rcr-2
17       l1/trove-rcr-3
18       l1/trove-rcr-4
19       l1/trove-rcr-5
20           l2/1d-conv
21        l2/1d-stencil
22      l2/2d-stencil-3
23    l2/mult-32-with-4
24           l3/1d-conv
25        l3/1d-stencil
26      l3/2d-stencil-3
27    l3/mult-32-with-4
28       l3/trove-crc-1
29       l3/trove-crc-2
30       l3/trove-crc-3
31       l3/trove-crc-4
32       l3/trove-crc-5
33    l3/trove-cr_sum-1
34    l3/trove-cr_sum-2
35    l3/trove-cr_sum-3
36    l3/trove-cr_sum-4
37    l3/trove-cr_sum-5
38       l3/trove-rcr-1
39       l3/trove-rcr-2
40       l3/trove-rcr-3
41       l3/trov

In [35]:
swinv_times_raw = {
    'l3/1d-conv': 13400,
    'l1/1d-conv': 8072,
    'l3/1d-stencil': 9233,
    'l1/1d-stencil': 4262,
    'l3/2d-stencil-5': 200545,
    'l1/2d-stencil-5': 36551,
    'l3/2d-stencil-3': 27267,
    'l1/2d-stencil-3': 8817,
    'l3/trove-crc-3': 10973,
    'l1/trove-crc-3': 627,
    'l3/trove-crc-5': 87108,
    'l1/trove-crc-5': 1328,
    'l3/trove-rcr-3': -1,
    'l1/trove-rcr-3': 2929,
    'l3/trove-rcr-5': -1,
    'l1/trove-rcr-5': 6307
}

In [36]:
swinv_times = pd.Series(swinv_times_raw).map(
lambda n: np.nan if n < 0 else n / 1000.0)

In [37]:
swinv_times

l3/1d-conv          13.400
l1/1d-conv           8.072
l3/1d-stencil        9.233
l1/1d-stencil        4.262
l3/2d-stencil-5    200.545
l1/2d-stencil-5     36.551
l3/2d-stencil-3     27.267
l1/2d-stencil-3      8.817
l3/trove-crc-3      10.973
l1/trove-crc-3       0.627
l3/trove-crc-5      87.108
l1/trove-crc-5       1.328
l3/trove-rcr-3         NaN
l1/trove-rcr-3       2.929
l3/trove-rcr-5         NaN
l1/trove-rcr-5       6.307
dtype: float64

In [38]:
# Append 5x5 stencil experiments
stencil_5_raw_data = fetch("2020-04-27-2d-stencil-5-timings")

In [39]:
stencil_5_raw = timing.process(stencil_5_raw_data)

In [40]:
swflow_totals = pd.concat([stencil_5_raw['total'],
                           swflow_times.set_index('spec')['total']])

In [41]:
times = pd.DataFrame({
    'Swizzleflow': swflow_totals[swinv_times.index.to_list()],
    'Swizzle Inventor': swinv_times
})
times

,Swizzleflow,Swizzle Inventor
spec,,
l3/1d-conv,2.920013,13.400
l1/1d-conv,2.580885,8.072
l3/1d-stencil,0.017842,9.233
l1/1d-stencil,0.012764,4.262
l3/2d-stencil-5,1325.083192,200.545
l1/2d-stencil-5,1403.734204,36.551
l3/2d-stencil-3,9.690038,27.267
l1/2d-stencil-3,9.313894,8.817
l3/trove-crc-3,15.421121,10.973


In [42]:
times = times.reset_index()
times['Level'] = times['spec'].map(lambda s: int(s[1]))
problems_map = {
    '1d-conv': 'Convolution (weights, k=3)',
    '1d-stencil': 'Convolution (no weights, k=3)',
    'trove-crc-3': 'Trove (CRC, s=3)',
    'trove-crc-5': 'Trove (CRC, s=5)',
    'trove-rcr-3': 'Trove (RCR, s=3)',
    'trove-rcr-5': 'Trove (RCR, s=5)',
    '2d-stencil-3': '2D stencil (k=3)',
    '2d-stencil-5': '2D stencil (k=5)'
}
times['Problem'] = times['spec'].map(lambda s: problems_map[s[3:]])
times

,spec,Swizzleflow,Swizzle Inventor,Level,Problem
0,l3/1d-conv,2.920013,13.400,3,"Convolution (weights, k=3)"
1,l1/1d-conv,2.580885,8.072,1,"Convolution (weights, k=3)"
2,l3/1d-stencil,0.017842,9.233,3,"Convolution (no weights, k=3)"
3,l1/1d-stencil,0.012764,4.262,1,"Convolution (no weights, k=3)"
4,l3/2d-stencil-5,1325.083192,200.545,3,2D stencil (k=5)
5,l1/2d-stencil-5,1403.734204,36.551,1,2D stencil (k=5)
6,l3/2d-stencil-3,9.690038,27.267,3,2D stencil (k=3)
7,l1/2d-stencil-3,9.313894,8.817,1,2D stencil (k=3)
8,l3/trove-crc-3,15.421121,10.973,3,"Trove (CRC, s=3)"
9,l1/trove-crc-3,14.586127,0.627,1,"Trove (CRC, s=3)"


In [43]:
times.drop(columns=['spec'], inplace=True)

In [44]:
fig, axs = plt.subplots(4, 2)
for idx, group in enumerate(times.groupby('Problem')):
    ax = axs[idx // 2][idx % 2]
    name, df = group
    df = df.set_index('Level')
    df.index = df.index.map({1: 'R', 3: 'F'})
    df = df.fillna(30 * 60) # Plot timeouts somehow
    df.plot(kind='barh', ax=ax, legend=False)
    ax.set_ylabel('')
    ax.set_xlabel("Time (s)")
    ax.set_title(name)
handles, labels = axs[3][1].get_legend_handles_labels()
fig.suptitle("Synthesis time comparisons")
fig.legend(handles, labels, loc='upper right')
fig.tight_layout()
fig.subplots_adjust(top=0.85, right=0.8)
fig.savefig('/tmp/comparisons.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
clear_names = times.copy()
clear_names['Level'] = clear_names['Level'].map({1: 'Restricted', 3: 'Full'})
table = clear_names.pivot(index='Problem', columns='Level', values=['Swizzle Inventor', 'Swizzleflow'])
table

Swizzle Inventor             Swizzleflow  \
Level                                     Full Restricted         Full   
Problem                                                                  
2D stencil (k=3)                        27.267      8.817     9.690038   
2D stencil (k=5)                       200.545     36.551  1325.083192   
Convolution (no weights, k=3)            9.233      4.262     0.017842   
Convolution (weights, k=3)              13.400      8.072     2.920013   
Trove (CRC, s=3)                        10.973      0.627    15.421121   
Trove (CRC, s=5)                        87.108      1.328   192.593073   
Trove (RCR, s=3)                           NaN      2.929    10.793976   
Trove (RCR, s=5)                           NaN      6.307   160.527233   

                                            
Level                           Restricted  
Problem                                     
2D stencil (k=3)                  9.313894  
2D stencil (k=5)               1403.734204  
Convolution (no weights, k=3)     0.012764  
Convolution (weights, k=3)        2.580885  
Trove (CRC, s=3)                 14.586127  
Trove (CRC, s=5)                208.811136  
Trove (RCR, s=3)                  9.839672  
Trove (RCR, s=5)                152.280725

In [46]:
c = table.columns
table = table[[c[3], c[2], c[1], c[0]]]
table

Swizzleflow              Swizzle Inventor  \
Level                           Restricted         Full       Restricted   
Problem                                                                    
2D stencil (k=3)                  9.313894     9.690038            8.817   
2D stencil (k=5)               1403.734204  1325.083192           36.551   
Convolution (no weights, k=3)     0.012764     0.017842            4.262   
Convolution (weights, k=3)        2.580885     2.920013            8.072   
Trove (CRC, s=3)                 14.586127    15.421121            0.627   
Trove (CRC, s=5)                208.811136   192.593073            1.328   
Trove (RCR, s=3)                  9.839672    10.793976            2.929   
Trove (RCR, s=5)                152.280725   160.527233            6.307   

                                        
Level                             Full  
Problem                                 
2D stencil (k=3)                27.267  
2D stencil (k=5)               200.545  
Convolution (no weights, k=3)    9.233  
Convolution (weights, k=3)      13.400  
Trove (CRC, s=3)                10.973  
Trove (CRC, s=5)                87.108  
Trove (RCR, s=3)                   NaN  
Trove (RCR, s=5)                   NaN

In [47]:
print(table.to_latex(na_rep = "---",
               float_format="{:0.0f}".format))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Swizzleflow} & \multicolumn{2}{l}{Swizzle Inventor} \\
Level &  Restricted & Full &       Restricted & Full \\
Problem                       &             &      &                  &      \\
\midrule
2D stencil (k=3)              &           9 &   10 &                9 &   27 \\
2D stencil (k=5)              &        1404 & 1325 &               37 &  201 \\
Convolution (no weights, k=3) &           0 &    0 &                4 &    9 \\
Convolution (weights, k=3)    &           3 &    3 &                8 &   13 \\
Trove (CRC, s=3)              &          15 &   15 &                1 &   11 \\
Trove (CRC, s=5)              &         209 &  193 &                1 &   87 \\
Trove (RCR, s=3)              &          10 &   11 &                3 &  nan \\
Trove (RCR, s=5)              &         152 &  161 &                6 &  nan \\
\bottomrule
\end{tabular}



In [48]:
## Old plot thoughs, retained in comments just in case
# plot_df = all_data[['points', 'spec']].copy()
# plot_df['In solution'] = all_data['n_solutions']
# plot_df['Not in solution'] = all_data['continued'] - all_data['n_solutions']
# plot_df.index.name = "Statement"
# fig = plt.figure()
# for idx, groupby_res in enumerate(plot_df.groupby(['points', 'spec'])):
#     name, data = groupby_res
#    points, spec = name
#     ax = fig.add_subplot(2, 2, idx + 1)
#     ax.set_title(f"{spec}, {points}")
#     ax.set_xlabel("Choices for array")
#     data.plot(kind='line', ax=ax)
# plot_df['Statement'] = plot_df.index
# df2 = plot_df[plot_df['spec'] == 'Trove']
# df2.set_index(['points', 'Statement']).drop('spec', axis=1)

In [49]:
# extraction.compute_basis_size(one_point)
# extraction.compute_basis_size(two_points)

In [50]:
# for test, df in itertools.chain(one_point.items(), two_points.items()):
#     df['redundant'] = (df['tested'] // df['basis_size']) - n_solutions[test]
#     df['rel_redundant'] = df['redundant'] / (df['tested'] // df['basis_size'])

In [51]:
# (one_point, two_points)

In [52]:
# redundancies = {}
# rel_redundancies = {}
# for k in n_solutions:
#     redundancies[k] = pd.DataFrame({'one point': one_point[k]['redundant'][2:-1],
#                                      'two points': two_points[k]['redundant'][2:-1]})
# 
#     rel_redundancies[k] = pd.DataFrame({'one point': one_point[k]['rel_redundant'][2:-1],
#                                         'two points': two_points[k]['rel_redundant'][2:-1]})
# redundancies

In [53]:
# pull_spec_in(redundancies).groupby('spec').plot()